`c++` and `openmp`

Na żywo rozmawialiśmy o funkcji `work_3b`

In [5]:
%%writefile fast.cpp
/*
<%
cfg['compiler_args'] = ['-std=c++17', '-fopenmp']
cfg['linker_args'] = ['-fopenmp']
setup_pybind11(cfg)
%>
*/

#include <math.h>
#include <tuple>
#include <vector>
#include <pybind11/pybind11.h>
//#include <pybind11/stl.h>
#include <pybind11/numpy.h>
#include <omp.h>

namespace py = pybind11;


std::tuple<double, double> work_2(const int kData_dim,
                                  const int kDivisions,
                                  py::array_t<int> &py_X0,
                                  py::array_t<int> &py_X1,
                                  const int kN_classes,
                                  py::array_t<double> &py_pseudo_counts,
                                  py::array_t<int> &py_y) {
    
    const int kC_Xdim = kDivisions + 1;
    const int kC_ydim = kN_classes;
    
    py::buffer_info py_X0_buf = py_X0.request();
    auto *X0 = static_cast<int *>(py_X0_buf.ptr);
    py::buffer_info py_X1_buf = py_X1.request();
    auto *X1 = static_cast<int *>(py_X1_buf.ptr);
    
    py::buffer_info py_y_buf = py_y.request();
    auto *y = static_cast<int *>(py_y_buf.ptr);
    
    py::buffer_info py_pseudo_counts_buf = py_pseudo_counts.request();
    auto *pseudo_counts = static_cast<double *>(py_pseudo_counts_buf.ptr);
    
    int contingency_m[kC_Xdim][kC_Xdim][kC_ydim] = {};
    double contingency_m_y[kC_Xdim][kC_Xdim] = {};
    
    for (int data_idx = 0; data_idx < kData_dim; data_idx++) {
        contingency_m[X0[data_idx]][X1[data_idx]][y[data_idx]]++;
    }
    
    double neg_H = 0., neg_H_X0 = 0., neg_H_X1 = 0.;
    
    #pragma omp parallel for reduction (+: neg_H, neg_H_X0, neg_H_X1)
    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; C_Xidx_i++) {
        for (int C_yidx = 0; C_yidx < kC_ydim; C_yidx++) {
            double count_X0 = 0;
            double count_X1 = 0;
            for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; C_Xidx_j++) {
                double count_ij = pseudo_counts[C_yidx] + contingency_m[C_Xidx_i][C_Xidx_j][C_yidx];
                count_X0 += pseudo_counts[C_yidx] + contingency_m[C_Xidx_j][C_Xidx_i][C_yidx];
                count_X1 += count_ij;
                contingency_m_y[C_Xidx_i][C_Xidx_j] += count_ij;
                neg_H += count_ij * log2(count_ij);
            }
            neg_H_X0 += count_X0 * log2(count_X0);
            neg_H_X1 += count_X1 * log2(count_X1);
        }
    }
    
    #pragma omp parallel for reduction (+: neg_H, neg_H_X0, neg_H_X1)
    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; C_Xidx_i++) {
        double count_X0_y = 0;
        double count_X1_y = 0;        
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; C_Xidx_j++) {
            neg_H -= contingency_m_y[C_Xidx_i][C_Xidx_j] * log2(contingency_m_y[C_Xidx_i][C_Xidx_j]);
            count_X0_y += contingency_m_y[C_Xidx_j][C_Xidx_i];
            count_X1_y += contingency_m_y[C_Xidx_i][C_Xidx_j];
        }
        neg_H_X0 -= count_X0_y * log2(count_X0_y);
        neg_H_X1 -= count_X1_y * log2(count_X1_y);
    }
    
    return {neg_H - neg_H_X0,
            neg_H - neg_H_X1
           };
}
    

std::tuple<double, double> work_2b(const int kData_dim,
                                  const int kDivisions,
                                  py::array_t<int> &py_X0,
                                  py::array_t<int> &py_X1,
                                  const int kN_classes,
                                  py::array_t<double> &py_pseudo_counts,
                                  py::array_t<int> &py_y) {
    
    const int kC_Xdim = kDivisions + 1;
    const int kC_ydim = kN_classes;
    
    py::buffer_info py_X0_buf = py_X0.request();
    auto *X0 = static_cast<int *>(py_X0_buf.ptr);
    py::buffer_info py_X1_buf = py_X1.request();
    auto *X1 = static_cast<int *>(py_X1_buf.ptr);
    
    py::buffer_info py_y_buf = py_y.request();
    auto *y = static_cast<int *>(py_y_buf.ptr);
    
    py::buffer_info py_pseudo_counts_buf = py_pseudo_counts.request();
    auto *pseudo_counts = static_cast<double *>(py_pseudo_counts_buf.ptr);
    
    int contingency_m[kC_Xdim + 1][kC_Xdim + 1][kC_ydim + 1] = {};
    

    for (int data_idx = 0; data_idx < kData_dim; data_idx++) {
        contingency_m[X0[data_idx]][X1[data_idx]][y[data_idx]]++;
        contingency_m[X0[data_idx]][X1[data_idx]][kC_ydim]++;
        contingency_m[X0[data_idx]][kC_Xdim][y[data_idx]]++;
        contingency_m[kC_Xdim][X1[data_idx]][y[data_idx]]++;
        contingency_m[X0[data_idx]][kC_Xdim][kC_ydim]++;
        contingency_m[kC_Xdim][X1[data_idx]][kC_ydim]++;
    }
    
    double neg_H = 0., neg_H_X0 = 0., neg_H_X1 = 0.;

    for (int C_Xidx_0 = 0; C_Xidx_0 < kC_Xdim; C_Xidx_0++) {
        for (int C_Xidx_1 = 0; C_Xidx_1 < kC_Xdim; C_Xidx_1++) {
          double count_y = contingency_m[C_Xidx_0][C_Xidx_1][kC_ydim];            
          for (int C_yidx = 0; C_yidx < kC_ydim; C_yidx++) {
            count_y += pseudo_counts[C_yidx];  
            double count = pseudo_counts[C_yidx] + contingency_m[C_Xidx_0][C_Xidx_1][C_yidx];
            neg_H += count * log2(count);  
          }
          neg_H -= count_y * log2(count_y);  
        }
    }

    for (int C_Xidx = 0; C_Xidx < kC_Xdim; C_Xidx++) {
        double count_X0_y = contingency_m[kC_Xdim][C_Xidx][kC_ydim];
        double count_X1_y = contingency_m[C_Xidx][kC_Xdim][kC_ydim];
        for (int C_yidx = 0; C_yidx < kC_ydim; C_yidx++) {
          count_X0_y += kC_Xdim * pseudo_counts[C_yidx];
          count_X1_y += kC_Xdim * pseudo_counts[C_yidx];
            
          double count_X0 = kC_Xdim * pseudo_counts[C_yidx] + contingency_m[kC_Xdim][C_Xidx][C_yidx];
          neg_H_X0 += count_X0 * log2(count_X0);
            
          double count_X1 = kC_Xdim * pseudo_counts[C_yidx] + contingency_m[C_Xidx][kC_Xdim][C_yidx];
          neg_H_X1 += count_X1 * log2(count_X1);
        }
        neg_H_X0 -= count_X0_y * log2(count_X0_y);
        neg_H_X1 -= count_X1_y * log2(count_X1_y);
    }

    return {neg_H - neg_H_X0, neg_H - neg_H_X1};
}


std::tuple<double, double, double> work_3z(const int kData_dim,
                                           const int kDivisions,
                                           py::array_t<int> &py_X0,
                                           py::array_t<int> &py_X1,
                                           py::array_t<int> &py_X2,
                                           const int kN_classes,
                                           py::array_t<double> &py_pseudo_counts,
                                           py::array_t<int> &py_y) {
    
    const int kC_Xdim = kDivisions + 1;
    const int kC_ydim = kN_classes;
    
    py::buffer_info py_X0_buf = py_X0.request();
    auto *X0 = static_cast<int *>(py_X0_buf.ptr);
    py::buffer_info py_X1_buf = py_X1.request();
    auto *X1 = static_cast<int *>(py_X1_buf.ptr);
    py::buffer_info py_X2_buf = py_X2.request();
    auto *X2 = static_cast<int *>(py_X2_buf.ptr);
    
    py::buffer_info py_y_buf = py_y.request();
    auto *y = static_cast<int *>(py_y_buf.ptr);
    
    py::buffer_info py_pseudo_counts_buf = py_pseudo_counts.request();
    auto *pseudo_counts = static_cast<double *>(py_pseudo_counts_buf.ptr);
    
    int contingency_m[kC_Xdim][kC_Xdim][kC_Xdim][kC_ydim] = {};
    double contingency_m_y[kC_Xdim][kC_Xdim][kC_Xdim] = {};

    for (int data_idx = 0; data_idx < kData_dim; data_idx++) {
        contingency_m[X0[data_idx]][X1[data_idx]][X2[data_idx]][y[data_idx]]++;
    }

    double neg_H = 0., neg_H_X0 = 0., neg_H_X1 = 0., neg_H_X2 = 0.;
    
    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; C_Xidx_i++) {
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; C_Xidx_j++) {
            for (int C_yidx = 0; C_yidx < kC_ydim; C_yidx++) {
                double count_X0 = 0;
                double count_X1 = 0;
                double count_X2 = 0;
                for (int C_Xidx_k = 0; C_Xidx_k < kC_Xdim; C_Xidx_k++) {
                    double count_ijk = pseudo_counts[C_yidx] + contingency_m[C_Xidx_i][C_Xidx_j][C_Xidx_k][C_yidx];
                    count_X0 += pseudo_counts[C_yidx] + contingency_m[C_Xidx_k][C_Xidx_i][C_Xidx_j][C_yidx];
                    count_X1 += pseudo_counts[C_yidx] + contingency_m[C_Xidx_i][C_Xidx_k][C_Xidx_j][C_yidx];
                    count_X2 += count_ijk;
                    contingency_m_y[C_Xidx_i][C_Xidx_j][C_Xidx_k] += count_ijk;
                    neg_H += count_ijk * log2(count_ijk);
                }
                neg_H_X0 += count_X0 * log2(count_X0);
                neg_H_X1 += count_X1 * log2(count_X1);
                neg_H_X2 += count_X2 * log2(count_X2);
            }
        }
    }

    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; C_Xidx_i++) {
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; C_Xidx_j++) {
            double count_X0_y = 0;
            double count_X1_y = 0;
            double count_X2_y = 0;
            for (int C_Xidx_k = 0; C_Xidx_k < kC_Xdim; C_Xidx_k++) {
                neg_H -= contingency_m_y[C_Xidx_i][C_Xidx_j][C_Xidx_k] * log2(contingency_m_y[C_Xidx_i][C_Xidx_j][C_Xidx_k]);
                count_X0_y += contingency_m_y[C_Xidx_k][C_Xidx_i][C_Xidx_j];
                count_X1_y += contingency_m_y[C_Xidx_i][C_Xidx_k][C_Xidx_j];
                count_X2_y += contingency_m_y[C_Xidx_i][C_Xidx_j][C_Xidx_k];
            }
            neg_H_X0 -= count_X0_y * log2(count_X0_y);
            neg_H_X1 -= count_X1_y * log2(count_X1_y);
            neg_H_X2 -= count_X2_y * log2(count_X2_y);
        }
    }

        
    return {neg_H - neg_H_X0,
            neg_H - neg_H_X1,
            neg_H - neg_H_X2
           };
}


std::tuple<double, double, double> work_3a(const int kData_dim,
                                           const int kDivisions,
                                           py::array_t<int> &py_X0,
                                           py::array_t<int> &py_X1,
                                           py::array_t<int> &py_X2,
                                           const int kN_classes,
                                           py::array_t<double> &py_pseudo_counts,
                                           py::array_t<int> &py_y) {
    
    const int kC_Xdim = kDivisions + 1;
    const int kC_ydim = kN_classes;
    
    py::buffer_info py_X0_buf = py_X0.request();
    auto *X0 = static_cast<int *>(py_X0_buf.ptr);
    py::buffer_info py_X1_buf = py_X1.request();
    auto *X1 = static_cast<int *>(py_X1_buf.ptr);
    py::buffer_info py_X2_buf = py_X2.request();
    auto *X2 = static_cast<int *>(py_X2_buf.ptr);
    
    py::buffer_info py_y_buf = py_y.request();
    auto *y = static_cast<int *>(py_y_buf.ptr);
    
    py::buffer_info py_pseudo_counts_buf = py_pseudo_counts.request();
    auto *pseudo_counts = static_cast<double *>(py_pseudo_counts_buf.ptr);
    
    int contingency_m[kC_Xdim][kC_Xdim][kC_Xdim][kC_ydim] = {};
    double contingency_m_y[kC_Xdim][kC_Xdim][kC_Xdim] = {};

    for (int data_idx = 0; data_idx < kData_dim; data_idx++) {
        contingency_m[X0[data_idx]][X1[data_idx]][X2[data_idx]][y[data_idx]]++;
    }

    double neg_H = 0., neg_H_X0 = 0., neg_H_X1 = 0., neg_H_X2 = 0.;
    
    #pragma omp parallel
    {
    
    #pragma omp for reduction (+: neg_H, neg_H_X0, neg_H_X1, neg_H_X2)
    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; C_Xidx_i++) {
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; C_Xidx_j++) {
            for (int C_yidx = 0; C_yidx < kC_ydim; C_yidx++) {
                double count_X0 = 0;
                double count_X1 = 0;
                double count_X2 = 0;
                for (int C_Xidx_k = 0; C_Xidx_k < kC_Xdim; C_Xidx_k++) {
                    double count_ijk = pseudo_counts[C_yidx] + contingency_m[C_Xidx_i][C_Xidx_j][C_Xidx_k][C_yidx];
                    count_X0 += pseudo_counts[C_yidx] + contingency_m[C_Xidx_k][C_Xidx_i][C_Xidx_j][C_yidx];
                    count_X1 += pseudo_counts[C_yidx] + contingency_m[C_Xidx_i][C_Xidx_k][C_Xidx_j][C_yidx];
                    count_X2 += count_ijk;
                    contingency_m_y[C_Xidx_i][C_Xidx_j][C_Xidx_k] += count_ijk;
                    neg_H += count_ijk * log2(count_ijk);
                }
                neg_H_X0 += count_X0 * log2(count_X0);
                neg_H_X1 += count_X1 * log2(count_X1);
                neg_H_X2 += count_X2 * log2(count_X2);
            }
        }
    }

    #pragma omp for reduction (+: neg_H, neg_H_X0, neg_H_X1, neg_H_X2)
    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; C_Xidx_i++) {
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; C_Xidx_j++) {
            double count_X0_y = 0;
            double count_X1_y = 0;
            double count_X2_y = 0;
            for (int C_Xidx_k = 0; C_Xidx_k < kC_Xdim; C_Xidx_k++) {
                neg_H -= contingency_m_y[C_Xidx_i][C_Xidx_j][C_Xidx_k] * log2(contingency_m_y[C_Xidx_i][C_Xidx_j][C_Xidx_k]);
                count_X0_y += contingency_m_y[C_Xidx_k][C_Xidx_i][C_Xidx_j];
                count_X1_y += contingency_m_y[C_Xidx_i][C_Xidx_k][C_Xidx_j];
                count_X2_y += contingency_m_y[C_Xidx_i][C_Xidx_j][C_Xidx_k];
            }
            neg_H_X0 -= count_X0_y * log2(count_X0_y);
            neg_H_X1 -= count_X1_y * log2(count_X1_y);
            neg_H_X2 -= count_X2_y * log2(count_X2_y);
        }
    }
    
    }
        
    return {neg_H - neg_H_X0,
            neg_H - neg_H_X1,
            neg_H - neg_H_X2
           };
}

    

std::tuple<double, double, double> work_3b(const int kData_dim,
                                           const int kDivisions,
                                           py::array_t<int> &py_X0,
                                           py::array_t<int> &py_X1,
                                           py::array_t<int> &py_X2,
                                           const int kN_classes,
                                           py::array_t<double> &py_pseudo_counts,
                                           py::array_t<int> &py_y) {
    
    const int kC_Xdim = kDivisions + 1;
    const int kC_ydim = kN_classes;
    
    py::buffer_info py_X0_buf = py_X0.request();
    auto *X0 = static_cast<int *>(py_X0_buf.ptr);
    py::buffer_info py_X1_buf = py_X1.request();
    auto *X1 = static_cast<int *>(py_X1_buf.ptr);
    py::buffer_info py_X2_buf = py_X2.request();
    auto *X2 = static_cast<int *>(py_X2_buf.ptr);
    
    py::buffer_info py_y_buf = py_y.request();
    auto *y = static_cast<int *>(py_y_buf.ptr);
    
    py::buffer_info py_pseudo_counts_buf = py_pseudo_counts.request();
    auto *pseudo_counts = static_cast<double *>(py_pseudo_counts_buf.ptr);
    
    int contingency_m[kC_Xdim][kC_Xdim][kC_Xdim][kC_ydim] = {};
    double contingency_m_y[kC_Xdim][kC_Xdim][kC_Xdim] = {};
        
    omp_lock_t contingency_m_lock[kC_Xdim][kC_Xdim][kC_Xdim][kC_ydim];
                                                                                                
    for (int C_Xidx_0 = 0; C_Xidx_0 < kC_Xdim; C_Xidx_0++) {
        for (int C_Xidx_1 = 0; C_Xidx_1 < kC_Xdim; C_Xidx_1++) {
            for (int C_Xidx_2 = 0; C_Xidx_2 < kC_Xdim; C_Xidx_2++) {
                for (int C_yidx = 0; C_yidx < kC_ydim; C_yidx++) {
                    omp_init_lock(&contingency_m_lock[C_Xidx_0][C_Xidx_1][C_Xidx_2][C_yidx]);
                }
            }
        }
    }
    
    double neg_H = 0., neg_H_X0 = 0., neg_H_X1 = 0., neg_H_X2 = 0.;
    
    #pragma omp parallel
    {
    
    #pragma omp for                                           
    for (int data_idx = 0; data_idx < kData_dim; data_idx++) {
        omp_set_lock(&contingency_m_lock[X0[data_idx]][X1[data_idx]][X2[data_idx]][y[data_idx]]);
        contingency_m[X0[data_idx]][X1[data_idx]][X2[data_idx]][y[data_idx]]++;
        omp_unset_lock(&contingency_m_lock[X0[data_idx]][X1[data_idx]][X2[data_idx]][y[data_idx]]);
    }
    
    #pragma omp for reduction (+: neg_H, neg_H_X0, neg_H_X1, neg_H_X2)
    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; C_Xidx_i++) {
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; C_Xidx_j++) {
            for (int C_yidx = 0; C_yidx < kC_ydim; C_yidx++) {
                double count_X0 = 0;
                double count_X1 = 0;
                double count_X2 = 0;
                for (int C_Xidx_k = 0; C_Xidx_k < kC_Xdim; C_Xidx_k++) {
                    double count_ijk = pseudo_counts[C_yidx] + contingency_m[C_Xidx_i][C_Xidx_j][C_Xidx_k][C_yidx];
                    count_X0 += pseudo_counts[C_yidx] + contingency_m[C_Xidx_k][C_Xidx_i][C_Xidx_j][C_yidx];
                    count_X1 += pseudo_counts[C_yidx] + contingency_m[C_Xidx_i][C_Xidx_k][C_Xidx_j][C_yidx];
                    count_X2 += count_ijk;
                    contingency_m_y[C_Xidx_i][C_Xidx_j][C_Xidx_k] += count_ijk;
                    neg_H += count_ijk * log2(count_ijk);
                }
                neg_H_X0 += count_X0 * log2(count_X0);
                neg_H_X1 += count_X1 * log2(count_X1);
                neg_H_X2 += count_X2 * log2(count_X2);
            }
        }
    }

    #pragma omp for reduction (+: neg_H, neg_H_X0, neg_H_X1, neg_H_X2)
    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; C_Xidx_i++) {
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; C_Xidx_j++) {
            double count_X0_y = 0;
            double count_X1_y = 0;
            double count_X2_y = 0;
            for (int C_Xidx_k = 0; C_Xidx_k < kC_Xdim; C_Xidx_k++) {
                neg_H -= contingency_m_y[C_Xidx_i][C_Xidx_j][C_Xidx_k] * log2(contingency_m_y[C_Xidx_i][C_Xidx_j][C_Xidx_k]);
                count_X0_y += contingency_m_y[C_Xidx_k][C_Xidx_i][C_Xidx_j];
                count_X1_y += contingency_m_y[C_Xidx_i][C_Xidx_k][C_Xidx_j];
                count_X2_y += contingency_m_y[C_Xidx_i][C_Xidx_j][C_Xidx_k];
            }
            neg_H_X0 -= count_X0_y * log2(count_X0_y);
            neg_H_X1 -= count_X1_y * log2(count_X1_y);
            neg_H_X2 -= count_X2_y * log2(count_X2_y);
        }
    }
    
    }
    
    for (int C_Xidx_0 = 0; C_Xidx_0 < kC_Xdim; C_Xidx_0++) {
        for (int C_Xidx_1 = 0; C_Xidx_1 < kC_Xdim; C_Xidx_1++) {
            for (int C_Xidx_2 = 0; C_Xidx_2 < kC_Xdim; C_Xidx_2++) {
                for (int C_yidx = 0; C_yidx < kC_ydim; C_yidx++) {
                    omp_destroy_lock(&contingency_m_lock[C_Xidx_0][C_Xidx_1][C_Xidx_2][C_yidx]);
                }
            }
        }
    }
        
    return {neg_H - neg_H_X0,
            neg_H - neg_H_X1,
            neg_H - neg_H_X2
           };
}

    
    
    
std::tuple<double, double, double> work_3c(const int kData_dim,
                                           const int kDivisions,
                                           py::array_t<int> &py_X0,
                                           py::array_t<int> &py_X1,
                                           py::array_t<int> &py_X2,
                                           const int kN_classes,
                                           py::array_t<double> &py_pseudo_counts,
                                           py::array_t<int> &py_y) {
    
    const int kC_Xdim = kDivisions + 1;
    const int kC_ydim = kN_classes;
    
    py::buffer_info py_X0_buf = py_X0.request();
    auto *X0 = static_cast<int *>(py_X0_buf.ptr);
    py::buffer_info py_X1_buf = py_X1.request();
    auto *X1 = static_cast<int *>(py_X1_buf.ptr);
    py::buffer_info py_X2_buf = py_X2.request();
    auto *X2 = static_cast<int *>(py_X2_buf.ptr);
    
    py::buffer_info py_pseudo_counts_buf = py_pseudo_counts.request();
    auto *pseudo_counts = (double *) py_pseudo_counts_buf.ptr;
    
    py::buffer_info py_y_buf = py_y.request();
    auto *y = (int *) py_y_buf.ptr;
    
    int contingency_m[kC_Xdim + 1][kC_Xdim + 1][kC_Xdim + 1][kC_ydim + 1] = {};

    for (int data_idx = 0; data_idx < kData_dim; data_idx++) {
        contingency_m[X0[data_idx]][X1[data_idx]][X2[data_idx]][y[data_idx]]++;
        contingency_m[X0[data_idx]][X1[data_idx]][X2[data_idx]][kC_ydim]++;
        contingency_m[X0[data_idx]][X1[data_idx]][kC_Xdim][y[data_idx]]++;
        contingency_m[X0[data_idx]][kC_Xdim][X2[data_idx]][y[data_idx]]++;
        contingency_m[kC_Xdim][X1[data_idx]][X2[data_idx]][y[data_idx]]++;
        contingency_m[X0[data_idx]][X1[data_idx]][kC_Xdim][kC_ydim]++;
        contingency_m[X0[data_idx]][kC_Xdim][X2[data_idx]][kC_ydim]++;
        contingency_m[kC_Xdim][X1[data_idx]][X2[data_idx]][kC_ydim]++;
    }
    
    double neg_H = 0., neg_H_X0 = 0., neg_H_X1 = 0., neg_H_X2 = 0.;
    
    #pragma omp parallel
    {
        
    #pragma omp for reduction (+: neg_H)
    for (int C_Xidx_0 = 0; C_Xidx_0 < kC_Xdim; C_Xidx_0++) {
        for (int C_Xidx_1 = 0; C_Xidx_1 < kC_Xdim; C_Xidx_1++) {
            for (int C_Xidx_2 = 0; C_Xidx_2 < kC_Xdim; C_Xidx_2++) {
              double count_y = contingency_m[C_Xidx_0][C_Xidx_1][C_Xidx_2][kC_ydim];            
              for (int C_yidx = 0; C_yidx < kC_ydim; C_yidx++) {
                count_y += pseudo_counts[C_yidx];  
                double count = pseudo_counts[C_yidx] + contingency_m[C_Xidx_0][C_Xidx_1][C_Xidx_2][C_yidx];
                neg_H += count * log2(count);  
              }
              neg_H -= count_y * log2(count_y);
            }
        }
    }

    #pragma omp for reduction (+: neg_H_X0, neg_H_X1, neg_H_X2)
    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; C_Xidx_i++) {
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; C_Xidx_j++) {
            double count_X0_y = contingency_m[kC_Xdim][C_Xidx_i][C_Xidx_j][kC_ydim];
            double count_X1_y = contingency_m[C_Xidx_i][kC_Xdim][C_Xidx_j][kC_ydim];
            double count_X2_y = contingency_m[C_Xidx_i][C_Xidx_j][kC_Xdim][kC_ydim];
            for (int C_yidx = 0; C_yidx < kC_ydim; C_yidx++) {
              count_X0_y += kC_Xdim * pseudo_counts[C_yidx];
              count_X1_y += kC_Xdim * pseudo_counts[C_yidx];
              count_X2_y += kC_Xdim * pseudo_counts[C_yidx];

              double count_X0 = kC_Xdim * pseudo_counts[C_yidx] + contingency_m[kC_Xdim][C_Xidx_i][C_Xidx_j][C_yidx];
              neg_H_X0 += count_X0 * log2(count_X0);

              double count_X1 = kC_Xdim * pseudo_counts[C_yidx] + contingency_m[C_Xidx_i][kC_Xdim][C_Xidx_j][C_yidx];
              neg_H_X1 += count_X1 * log2(count_X1);
                
              double count_X2 = kC_Xdim * pseudo_counts[C_yidx] + contingency_m[C_Xidx_i][C_Xidx_j][kC_Xdim][C_yidx];
              neg_H_X2 += count_X2 * log2(count_X2);
            }
            neg_H_X0 -= count_X0_y * log2(count_X0_y);
            neg_H_X1 -= count_X1_y * log2(count_X1_y);
            neg_H_X2 -= count_X2_y * log2(count_X2_y);
        }
    }
    
    }
    
    return {neg_H - neg_H_X0, neg_H - neg_H_X1, neg_H - neg_H_X2};
}

    
std::tuple<double, double, double> work_3d(const int kData_dim,
                                           const int kDivisions,
                                           py::array_t<int> &py_X0,
                                           py::array_t<int> &py_X1,
                                           py::array_t<int> &py_X2,
                                           const int kN_classes,
                                           py::array_t<double> &py_pseudo_counts,
                                           py::array_t<int> &py_y) {
    
    const int kC_Xdim = kDivisions + 1;
    const int kC_ydim = kN_classes;
    
    py::buffer_info py_X0_buf = py_X0.request();
    auto *X0 = static_cast<int *>(py_X0_buf.ptr);
    py::buffer_info py_X1_buf = py_X1.request();
    auto *X1 = static_cast<int *>(py_X1_buf.ptr);
    py::buffer_info py_X2_buf = py_X2.request();
    auto *X2 = static_cast<int *>(py_X2_buf.ptr);
    
    py::buffer_info py_pseudo_counts_buf = py_pseudo_counts.request();
    auto *pseudo_counts = (double *) py_pseudo_counts_buf.ptr;
    
    py::buffer_info py_y_buf = py_y.request();
    auto *y = (int *) py_y_buf.ptr;
    
    int contingency_m[kC_Xdim][kC_Xdim][kC_Xdim][kC_ydim] = {};
    for (int data_idx = 0; data_idx < kData_dim; data_idx++) {
        contingency_m[X0[data_idx]][X1[data_idx]][X2[data_idx]][y[data_idx]]++;
    }
    
    double neg_H = 0., neg_H_X0 = 0., neg_H_X1 = 0., neg_H_X2 = 0.;
    
    #pragma omp parallel
    {
    #pragma omp sections
    {
        #pragma omp section
        {
        double local_neg_H = 0.;
        for (int C_Xidx_0 = 0; C_Xidx_0 < kC_Xdim; C_Xidx_0++) {
            for (int C_Xidx_1 = 0; C_Xidx_1 < kC_Xdim; C_Xidx_1++) {
                for (int C_Xidx_2 = 0; C_Xidx_2 < kC_Xdim; C_Xidx_2++) {
                    double count_y = 0.;
                    for (int C_yidx = 0; C_yidx < kC_ydim; C_yidx++) {
                        double count = pseudo_counts[C_yidx] + contingency_m[C_Xidx_0][C_Xidx_1][C_Xidx_2][C_yidx];
                        local_neg_H += count * log2(count);
                        count_y += count;
                    }
                    local_neg_H -= count_y * log2(count_y);
                }
            }
        }
        #pragma omp atomic
            neg_H += local_neg_H;
        }        
        
        #pragma omp section
        {
        double local_neg_H_X0 = 0.;
        for (int C_Xidx_1 = 0; C_Xidx_1 < kC_Xdim; C_Xidx_1++) {
            for (int C_Xidx_2 = 0; C_Xidx_2 < kC_Xdim; C_Xidx_2++) {
                double count_y_X0 = 0.;
                for (int C_yidx = 0; C_yidx < kC_ydim; C_yidx++) {
                    double count_X0 = 0.;
                    for (int C_Xidx_0 = 0; C_Xidx_0 < kC_Xdim; C_Xidx_0++) {
                        double count = pseudo_counts[C_yidx] + contingency_m[C_Xidx_0][C_Xidx_1][C_Xidx_2][C_yidx];
                        count_X0 += count;
                    }
                    local_neg_H_X0 += count_X0 * log2(count_X0);
                    count_y_X0 += count_X0;
                }
                local_neg_H_X0 -= count_y_X0 * log2(count_y_X0);
            }
        }
        #pragma omp atomic
            neg_H_X0 += local_neg_H_X0;
        }
        
        #pragma omp section
        {
        double local_neg_H_X1 = 0.;
        for (int C_Xidx_0 = 0; C_Xidx_0 < kC_Xdim; C_Xidx_0++) {
            for (int C_Xidx_2 = 0; C_Xidx_2 < kC_Xdim; C_Xidx_2++) {
                double count_y_X1 = 0.;
                for (int C_yidx = 0; C_yidx < kC_ydim; C_yidx++) {
                    double count_X1 = 0.;
                    for (int C_Xidx_1 = 0; C_Xidx_1 < kC_Xdim; C_Xidx_1++) {
                        double count = pseudo_counts[C_yidx] + contingency_m[C_Xidx_0][C_Xidx_1][C_Xidx_2][C_yidx];
                        count_X1 += count;
                    }
                    local_neg_H_X1 += count_X1 * log2(count_X1);
                    count_y_X1 += count_X1;
                }
                local_neg_H_X1 -= count_y_X1 * log2(count_y_X1);
            }
        }
        #pragma omp atomic
            neg_H_X1 += local_neg_H_X1;
        }
        
        #pragma omp section
        {
        double local_neg_H_X2 = 0.;
        for (int C_Xidx_0 = 0; C_Xidx_0 < kC_Xdim; C_Xidx_0++) {
            for (int C_Xidx_1 = 0; C_Xidx_1 < kC_Xdim; C_Xidx_1++) {
                double count_y_X2 = 0.;
                for (int C_yidx = 0; C_yidx < kC_ydim; C_yidx++) {
                    double count_X2 = 0.;
                    for (int C_Xidx_2 = 0; C_Xidx_2 < kC_Xdim; C_Xidx_2++) {
                        double count = pseudo_counts[C_yidx] + contingency_m[C_Xidx_0][C_Xidx_1][C_Xidx_2][C_yidx];
                        count_X2 += count;
                    }
                    local_neg_H_X2 += count_X2 * log2(count_X2);
                    count_y_X2 += count_X2;
                }
                local_neg_H_X2 -= count_y_X2 * log2(count_y_X2);
            }
        }
        #pragma omp atomic
            neg_H_X2 += local_neg_H_X2;
        }        
    }
    }
    
    return {neg_H - neg_H_X0, neg_H - neg_H_X1, neg_H - neg_H_X2};
}

    
    
std::tuple<double, double, double> work_3e(const int kData_dim,
                                           const int kDivisions,
                                           py::array_t<int> &py_X0,
                                           py::array_t<int> &py_X1,
                                           py::array_t<int> &py_X2,
                                           const int kN_classes,
                                           py::array_t<double> &py_pseudo_counts,
                                           py::array_t<int> &py_y) {
    
    const int kC_Xdim = kDivisions + 1;
    const int kC_ydim = kN_classes;
    
    py::buffer_info py_X0_buf = py_X0.request();
    auto *X0 = static_cast<int *>(py_X0_buf.ptr);
    py::buffer_info py_X1_buf = py_X1.request();
    auto *X1 = static_cast<int *>(py_X1_buf.ptr);
    py::buffer_info py_X2_buf = py_X2.request();
    auto *X2 = static_cast<int *>(py_X2_buf.ptr);

    py::buffer_info py_pseudo_counts_buf = py_pseudo_counts.request();
    auto *pseudo_counts = static_cast<double *>(py_pseudo_counts_buf.ptr);
    
    double pseudo_counts_y = 0.0;
    for (int i = 0; i < kN_classes; i++) pseudo_counts_y += pseudo_counts[i];
    
    py::buffer_info py_y_buf = py_y.request();
    auto *y = static_cast<int *>(py_y_buf.ptr);
                                                 
    int contingency_m[kC_Xdim][kC_Xdim][kC_Xdim][kC_ydim + 1] = {};
    
    for (int data_idx = 0; data_idx < kData_dim; data_idx++) {
        contingency_m[X0[data_idx]][X1[data_idx]][X2[data_idx]][y[data_idx]]++;
        contingency_m[X0[data_idx]][X1[data_idx]][X2[data_idx]][kC_ydim]++;
    }    

    
    double neg_H = 0., neg_H_X0 = 0., neg_H_X1 = 0., neg_H_X2 = 0.;
    
    #pragma omp parallel
    {
    
    #pragma omp for reduction (+: neg_H, neg_H_X0, neg_H_X1, neg_H_X2)
    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; C_Xidx_i++) {
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; C_Xidx_j++) {
            for (int C_yidx = 0; C_yidx < kC_ydim; C_yidx++) {
                double count_X0 = 0;
                double count_X1 = 0;
                double count_X2 = 0;
                for (int C_Xidx_k = 0; C_Xidx_k < kC_Xdim; C_Xidx_k++) {
                    double count_ijk = pseudo_counts[C_yidx] + contingency_m[C_Xidx_i][C_Xidx_j][C_Xidx_k][C_yidx];
                    
                    count_X0 += pseudo_counts[C_yidx] + contingency_m[C_Xidx_k][C_Xidx_i][C_Xidx_j][C_yidx];
                    count_X1 += pseudo_counts[C_yidx] + contingency_m[C_Xidx_i][C_Xidx_k][C_Xidx_j][C_yidx];
                    count_X2 += count_ijk;
                    
                    neg_H += count_ijk * log2(count_ijk);
                }
                neg_H_X0 += count_X0 * log2(count_X0);
                neg_H_X1 += count_X1 * log2(count_X1);
                neg_H_X2 += count_X2 * log2(count_X2);
            }
        }
    }

    #pragma omp for reduction (+: neg_H, neg_H_X0, neg_H_X1, neg_H_X2)
    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; C_Xidx_i++) {
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; C_Xidx_j++) {
            double count_X0_y = 0;
            double count_X1_y = 0;
            double count_X2_y = 0;
            for (int C_Xidx_k = 0; C_Xidx_k < kC_Xdim; C_Xidx_k++) {
                double count_y_ijkl = pseudo_counts_y + contingency_m[C_Xidx_i][C_Xidx_j][C_Xidx_k][kC_ydim];
                
                count_X0_y += pseudo_counts_y + contingency_m[C_Xidx_k][C_Xidx_i][C_Xidx_j][kC_ydim];
                count_X1_y += pseudo_counts_y + contingency_m[C_Xidx_i][C_Xidx_k][C_Xidx_j][kC_ydim];
                count_X2_y += count_y_ijkl;

                neg_H -= count_y_ijkl * log2(count_y_ijkl);
            }
            neg_H_X0 -= count_X0_y * log2(count_X0_y);
            neg_H_X1 -= count_X1_y * log2(count_X1_y);
            neg_H_X2 -= count_X2_y * log2(count_X2_y);
        }
    }
        
    }
    
    return {neg_H - neg_H_X0,
            neg_H - neg_H_X1,
            neg_H - neg_H_X2
           };
}
    


    
std::tuple<double, double, double> work_3f(const int kData_dim,
                                           const int kDivisions,
                                           py::array_t<int> &py_X0,
                                           py::array_t<int> &py_X1,
                                           py::array_t<int> &py_X2,
                                           const int kN_classes,
                                           py::array_t<double> &py_pseudo_counts,
                                           py::array_t<int> &py_y) {
    
    const int kC_Xdim = kDivisions + 1;
    const int kC_ydim = kN_classes;
    
    py::buffer_info py_X0_buf = py_X0.request();
    auto *X0 = static_cast<int *>(py_X0_buf.ptr);
    py::buffer_info py_X1_buf = py_X1.request();
    auto *X1 = static_cast<int *>(py_X1_buf.ptr);
    py::buffer_info py_X2_buf = py_X2.request();
    auto *X2 = static_cast<int *>(py_X2_buf.ptr);

    py::buffer_info py_pseudo_counts_buf = py_pseudo_counts.request();
    auto *pseudo_counts = static_cast<double *>(py_pseudo_counts_buf.ptr);
    
    double pseudo_counts_y = 0.0;
    for (int i = 0; i < kN_classes; i++) pseudo_counts_y += pseudo_counts[i];
    
    py::buffer_info py_y_buf = py_y.request();
    auto *y = static_cast<int *>(py_y_buf.ptr);
                                                 
    int contingency_m[kC_Xdim][kC_Xdim][kC_Xdim][kC_ydim + 1] = {};
    
    omp_lock_t contingency_m_lock[kC_Xdim][kC_Xdim][kC_Xdim];
                                                                                                
    for (int C_Xidx_0 = 0; C_Xidx_0 < kC_Xdim; C_Xidx_0++) {
        for (int C_Xidx_1 = 0; C_Xidx_1 < kC_Xdim; C_Xidx_1++) {
            for (int C_Xidx_2 = 0; C_Xidx_2 < kC_Xdim; C_Xidx_2++) {
                omp_init_lock(&contingency_m_lock[C_Xidx_0][C_Xidx_1][C_Xidx_2]);
            }
        }
    }
    
    double neg_H = 0., neg_H_X0 = 0., neg_H_X1 = 0., neg_H_X2 = 0.;
    
    #pragma omp parallel
    {
        
    #pragma omp for                                           
    for (int data_idx = 0; data_idx < kData_dim; data_idx++) {
        omp_set_lock(&contingency_m_lock[X0[data_idx]][X1[data_idx]][X2[data_idx]]);
        contingency_m[X0[data_idx]][X1[data_idx]][X2[data_idx]][y[data_idx]]++;
        contingency_m[X0[data_idx]][X1[data_idx]][X2[data_idx]][kC_ydim]++;
        omp_unset_lock(&contingency_m_lock[X0[data_idx]][X1[data_idx]][X2[data_idx]]);
    }
    
    #pragma omp for reduction (+: neg_H, neg_H_X0, neg_H_X1, neg_H_X2)
    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; C_Xidx_i++) {
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; C_Xidx_j++) {
            for (int C_yidx = 0; C_yidx < kC_ydim; C_yidx++) {
                double count_X0 = 0;
                double count_X1 = 0;
                double count_X2 = 0;
                for (int C_Xidx_k = 0; C_Xidx_k < kC_Xdim; C_Xidx_k++) {
                    double count_ijk = pseudo_counts[C_yidx] + contingency_m[C_Xidx_i][C_Xidx_j][C_Xidx_k][C_yidx];
                    
                    count_X0 += pseudo_counts[C_yidx] + contingency_m[C_Xidx_k][C_Xidx_i][C_Xidx_j][C_yidx];
                    count_X1 += pseudo_counts[C_yidx] + contingency_m[C_Xidx_i][C_Xidx_k][C_Xidx_j][C_yidx];
                    count_X2 += count_ijk;
                    
                    neg_H += count_ijk * log2(count_ijk);
                }
                neg_H_X0 += count_X0 * log2(count_X0);
                neg_H_X1 += count_X1 * log2(count_X1);
                neg_H_X2 += count_X2 * log2(count_X2);
            }
        }
    }

    #pragma omp for reduction (+: neg_H, neg_H_X0, neg_H_X1, neg_H_X2)
    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; C_Xidx_i++) {
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; C_Xidx_j++) {
            double count_X0_y = 0;
            double count_X1_y = 0;
            double count_X2_y = 0;
            for (int C_Xidx_k = 0; C_Xidx_k < kC_Xdim; C_Xidx_k++) {
                double count_y_ijkl = pseudo_counts_y + contingency_m[C_Xidx_i][C_Xidx_j][C_Xidx_k][kC_ydim];
                
                count_X0_y += pseudo_counts_y + contingency_m[C_Xidx_k][C_Xidx_i][C_Xidx_j][kC_ydim];
                count_X1_y += pseudo_counts_y + contingency_m[C_Xidx_i][C_Xidx_k][C_Xidx_j][kC_ydim];
                count_X2_y += count_y_ijkl;

                neg_H -= count_y_ijkl * log2(count_y_ijkl);
            }
            neg_H_X0 -= count_X0_y * log2(count_X0_y);
            neg_H_X1 -= count_X1_y * log2(count_X1_y);
            neg_H_X2 -= count_X2_y * log2(count_X2_y);
        }
    }
        
    }
    
    for (int C_Xidx_0 = 0; C_Xidx_0 < kC_Xdim; C_Xidx_0++) {
        for (int C_Xidx_1 = 0; C_Xidx_1 < kC_Xdim; C_Xidx_1++) {
            for (int C_Xidx_2 = 0; C_Xidx_2 < kC_Xdim; C_Xidx_2++) {
                omp_destroy_lock(&contingency_m_lock[C_Xidx_0][C_Xidx_1][C_Xidx_2]);
            }
        }
    }
    
    return {neg_H - neg_H_X0,
            neg_H - neg_H_X1,
            neg_H - neg_H_X2
           };
}


std::tuple<double, double, double> work_3g(const int kData_dim,
                                           const int kDivisions,
                                           py::array_t<int> &py_X0,
                                           py::array_t<int> &py_X1,
                                           py::array_t<int> &py_X2,
                                           const int kN_classes,
                                           py::array_t<double> &py_pseudo_counts,
                                           py::array_t<int> &py_y) {
    
    const int kC_Xdim = kDivisions + 1;
    const int kC_ydim = kN_classes;
    
    py::buffer_info py_X0_buf = py_X0.request();
    auto *X0 = static_cast<int *>(py_X0_buf.ptr);
    py::buffer_info py_X1_buf = py_X1.request();
    auto *X1 = static_cast<int *>(py_X1_buf.ptr);
    py::buffer_info py_X2_buf = py_X2.request();
    auto *X2 = static_cast<int *>(py_X2_buf.ptr);
    
    py::buffer_info py_y_buf = py_y.request();
    auto *y = static_cast<int *>(py_y_buf.ptr);
    
    py::buffer_info py_pseudo_counts_buf = py_pseudo_counts.request();
    auto *pseudo_counts = static_cast<double *>(py_pseudo_counts_buf.ptr);
    
    int contingency_m[kC_Xdim][kC_Xdim][kC_Xdim][kC_ydim] = {};
    double contingency_m_y[kC_Xdim][kC_Xdim][kC_Xdim] = {};
    
    #pragma omp parallel for
    for (int data_idx = 0; data_idx < kData_dim; data_idx++) {
        #pragma omp atomic
            contingency_m[X0[data_idx]][X1[data_idx]][X2[data_idx]][y[data_idx]]++;
    }

    double neg_H = 0., neg_H_X0 = 0., neg_H_X1 = 0., neg_H_X2 = 0.;
    
    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; C_Xidx_i++) {
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; C_Xidx_j++) {
            for (int C_yidx = 0; C_yidx < kC_ydim; C_yidx++) {
                double count_X0 = 0;
                double count_X1 = 0;
                double count_X2 = 0;
                for (int C_Xidx_k = 0; C_Xidx_k < kC_Xdim; C_Xidx_k++) {
                    double count_ijk = pseudo_counts[C_yidx] + contingency_m[C_Xidx_i][C_Xidx_j][C_Xidx_k][C_yidx];
                    count_X0 += pseudo_counts[C_yidx] + contingency_m[C_Xidx_k][C_Xidx_i][C_Xidx_j][C_yidx];
                    count_X1 += pseudo_counts[C_yidx] + contingency_m[C_Xidx_i][C_Xidx_k][C_Xidx_j][C_yidx];
                    count_X2 += count_ijk;
                    contingency_m_y[C_Xidx_i][C_Xidx_j][C_Xidx_k] += count_ijk;
                    neg_H += count_ijk * log2(count_ijk);
                }
                neg_H_X0 += count_X0 * log2(count_X0);
                neg_H_X1 += count_X1 * log2(count_X1);
                neg_H_X2 += count_X2 * log2(count_X2);
            }
        }
    }

    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; C_Xidx_i++) {
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; C_Xidx_j++) {
            double count_X0_y = 0;
            double count_X1_y = 0;
            double count_X2_y = 0;
            for (int C_Xidx_k = 0; C_Xidx_k < kC_Xdim; C_Xidx_k++) {
                neg_H -= contingency_m_y[C_Xidx_i][C_Xidx_j][C_Xidx_k] * log2(contingency_m_y[C_Xidx_i][C_Xidx_j][C_Xidx_k]);
                count_X0_y += contingency_m_y[C_Xidx_k][C_Xidx_i][C_Xidx_j];
                count_X1_y += contingency_m_y[C_Xidx_i][C_Xidx_k][C_Xidx_j];
                count_X2_y += contingency_m_y[C_Xidx_i][C_Xidx_j][C_Xidx_k];
            }
            neg_H_X0 -= count_X0_y * log2(count_X0_y);
            neg_H_X1 -= count_X1_y * log2(count_X1_y);
            neg_H_X2 -= count_X2_y * log2(count_X2_y);
        }
    }

        
    return {neg_H - neg_H_X0,
            neg_H - neg_H_X1,
            neg_H - neg_H_X2
           };
}
        

PYBIND11_MODULE(fast, module) {
    module.def("work_2", &work_2);
    module.def("work_2b", &work_2b);
    module.def("work_3z", &work_3z);
    module.def("work_3a", &work_3a);
    module.def("work_3b", &work_3b);
    module.def("work_3c", &work_3c);
    module.def("work_3d", &work_3d);
    module.def("work_3e", &work_3e);
    module.def("work_3f", &work_3f);
    module.def("work_3g", &work_3g);
}

Overwriting fast.cpp


Import the `cpp` module to python (it's now that the compilation happens, assuming there were some changes...)

(Remark: ...but one has to restart the kernel to re-import a module, even if its code has been modified.) 

In [9]:
import cppimport
#cppimport.set_quiet(False)

fast = cppimport.imp("fast")

---
Make a data sample: pick `n` last X-columns in the `madelon`

In [2]:
n = 50

from pandas import read_csv

madelon = read_csv("madelon.csv", header=None)
madelon.iloc[:,-(n + 2):].to_csv('madelon_tiny.csv', header=False, index=False)
madelon_tiny = read_csv("madelon_tiny.csv", dtype = 'float64', header=None)
madelon_tiny.shape

(2000, 52)

---
Run the thing synchronously (make it a standalone script for profiling)

In [10]:
%%writefile february0_n1.py
import csv
import numpy as np
from scipy.stats import rankdata
from pandas import read_csv, DataFrame
from itertools import product, chain, starmap, combinations, combinations_with_replacement
from time import time
import pickle

import fast

time0 = time()

k = 2
divisions = 2
range_ = 0.00
seed = 123

# 1. Function definitions

def discretize(seq, divisions=divisions, range_=range_, seed=seed):
    '''
    >>> discretize([3, 4, 1, 8, 13, 8], divisions=4, range_=0, seed=123) = array([1, 1, 0, 2, 3, 2])
    where
    ranks = [2., 3., 1., 4.5, 6., 4.5]
    tresholds = [1.5,  3.,  4.5]
    '''
    np.random.seed(seed)
    ranks = rankdata(seq, method='ordinal') # method='ordinal'/'average' ?

    random_blocks = np.cumsum(range_ * (2 * np.random.random(divisions + 1) - 1) + np.ones(divisions + 1))
    tresholds = random_blocks[:-1] / random_blocks[-1] * len(seq)
    
    discrete_seq = np.zeros(len(seq), dtype='int32')
    for treshold in tresholds:
        discrete_seq[ranks > treshold] += 1
    return discrete_seq

discretize_vec = np.vectorize(discretize, signature='(n)->(n)', excluded=['divisions', 'range_', 'seed'])

# 2. Read the data

file = "madelon.csv"
input_ = []
with open(file) as csvfile:
    reader = csv.reader(csvfile, delimiter=',',
                        quoting=csv.QUOTE_NONNUMERIC)
    for row in reader:
        input_.append(row)
        
input_ = np.array(input_, dtype='float64').T[:-1]
data = np.empty(input_.shape, dtype='int32')
data[:-1] = discretize_vec(input_[:-1])
data[-1] = input_[-1:].astype('int32')

labels, counts = np.unique(data[-1], return_counts=True)
n_classes = len(labels)

xi = 0.25
pseudo_counts = xi * counts / np.min(counts)

dim0, dim1 = data[:-1].shape

# 3. More function definitions

def tuple_generator(k=k, dim0=dim0):
    '''
    Python-generator.
    E.g. output for k=2:
    {0,1}, {0,2}, ..., {0, dim0-1}, {1,2}, ..., {1,dim0-1}, ..., {dim0-2, dim0-1}
    Go with combinations_with_replacement() to include diagonal tuples like {0, 0}
    '''        
    return combinations(range(dim0), k)    

def neg_H(p):
    return p * np.log2(p)

def neg_H_cond(matrix):
    return np.sum(neg_H(matrix)) - np.sum(neg_H(np.sum(matrix, axis=-1)))

def slow_work(indeces):
    '''
    indeces -> tuple
    Work-function.
    Output: tuple of Information Gains implicitly corresponding to the indeces
    '''
    # contingency-matrix: begin with pseudo-counts
    contingency_m = np.empty([divisions + 1] * k + [len(labels)], dtype='float64')
    for label, pseudo_count in enumerate(pseudo_counts):
        contingency_m[..., label] = pseudo_count
    
    # contingency-matrix: normal counts
    for c_index in data[list(indeces) + [-1]].T:
        contingency_m[tuple(c_index)] += 1
    
    results = []
    for i in range(len(indeces)):
        result = neg_H_cond(contingency_m) - neg_H_cond(np.sum(contingency_m, axis=i))
        results.append(result)
    return tuple(results)


def record(tuple_, IGs, records):
    for column, IG in zip(tuple_, IGs):
        if column not in records or IG > records[column][0]:
            records[column] = (IG, tuple_)

final_results = {}

for tuple_ in tuple_generator():
    #IGs = slow_work(tuple_)
    IGs = fast.work_2(dim1, divisions, data[tuple_[0]], data[tuple_[1]], n_classes, pseudo_counts, data[-1])
    #IGs = fast.work_2b(dim1, divisions, data[tuple_[0]], data[tuple_[1]], n_classes, pseudo_counts, data[-1])
    #IGs = fast.work_3z(dim1, divisions, data[tuple_[0]], data[tuple_[1]], data[tuple_[2]], n_classes, pseudo_counts, data[-1])
    #IGs = fast.work_3a(dim1, divisions, data[tuple_[0]], data[tuple_[1]], data[tuple_[2]], n_classes, pseudo_counts, data[-1])
    #IGs = fast.work_3b(dim1, divisions, data[tuple_[0]], data[tuple_[1]], data[tuple_[2]], n_classes, pseudo_counts, data[-1])
    #IGs = fast.work_3c(dim1, divisions, data[tuple_[0]], data[tuple_[1]], data[tuple_[2]], n_classes, pseudo_counts, data[-1])
    #IGs = fast.work_3d(dim1, divisions, data[tuple_[0]], data[tuple_[1]], data[tuple_[2]], n_classes, pseudo_counts, data[-1])
    #IGs = fast.work_3e(dim1, divisions, data[tuple_[0]], data[tuple_[1]], data[tuple_[2]], n_classes, pseudo_counts, data[-1])
    #IGs = fast.work_3f(dim1, divisions, data[tuple_[0]], data[tuple_[1]], data[tuple_[2]], n_classes, pseudo_counts, data[-1])
    #IGs = fast.work_3g(dim1, divisions, data[tuple_[0]], data[tuple_[1]], data[tuple_[2]], n_classes, pseudo_counts, data[-1])
    record(tuple_, IGs, final_results)

# result
print("Finished in", time() - time0, "sec.")

with open("february0_n1_results.pkl", "wb") as file:
    pickle.dump(final_results, file)

Overwriting february0_n1.py


In [11]:
%run february0_n1.py

Finished in 3.2627100944519043 sec.


In [12]:
import pandas as pd
import pickle
with open("february0_n1_results.pkl", "rb") as file:
    final_results = pickle.load(file)

result_n1_df = pd.DataFrame(final_results).T.rename(columns={0: 'IG_max', 1: 'tuple'})
result_n1_df.sort_values('IG_max', ascending=False)[:10]

,IG_max,tuple
241,140.978,"(241, 472)"
475,135.357,"(472, 475)"
336,133.608,"(336, 472)"
64,132.514,"(64, 472)"
338,118.192,"(48, 338)"
128,107.074,"(128, 442)"
105,105.692,"(105, 442)"
472,105.306,"(336, 472)"
442,97.847,"(336, 442)"
48,95.377,"(48, 433)"


Some profiling

In [49]:
%%bash
python -m cProfile -o february0_n1.prof february0_n1.py

Finished in 0.38264942169189453 sec.


In [50]:
import pstats
p = pstats.Stats('february0_n1.prof')
#p.sort_stats('time').print_stats(15)
p.strip_dirs().sort_stats('time').print_stats(10)

Fri Feb 22 21:11:22 2019    february0_n1.prof

         469874 function calls (459564 primitive calls) in 0.999 seconds

   Ordered by: internal time
   List reduced from 3425 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     4060    0.321    0.000    0.321    0.000 {built-in method fast.work_3a}
      539    0.070    0.000    0.070    0.000 {built-in method marshal.loads}
    822/1    0.047    0.000    1.002    1.002 {built-in method builtins.exec}
   126/91    0.043    0.000    0.084    0.001 {built-in method _imp.create_dynamic}
        1    0.035    0.035    1.002    1.002 february0_n1.py:1(<module>)
1574/1559    0.030    0.000    0.085    0.000 {built-in method builtins.__build_class__}
      297    0.022    0.000    0.055    0.000 doccer.py:12(docformat)
       48    0.014    0.000    0.015    0.000 config.py:414(register_option)
      870    0.014    0.000    0.014    0.000 {method 'sub' of '_sre.SRE_Pattern' objects}
   

---
Make a script that uses mpi4py and run with mpi from bash

In [55]:
%%writefile february0_mpi.py
import csv
import numpy as np
from scipy.stats import rankdata
from itertools import product, chain, starmap, combinations, combinations_with_replacement
import pickle

import fast

from mpi4py import MPI
comm = MPI.COMM_WORLD
comm.Barrier()
time0 = MPI.Wtime()
size = comm.Get_size()
rank = comm.Get_rank()

k = 3
window = 5
divisions = 10
range_ = 0.0
seed = 123
    

# 1. Function definitions

def discretize(seq, divisions=divisions, range_=range_, seed=seed):
    '''
    >>> discretize([3, 4, 1, 8, 13, 8], divisions=4, range_=0, seed=123) = array([1, 1, 0, 2, 3, 2])
    where
    ranks = [2., 3., 1., 4.5, 6., 4.5]
    tresholds = [1.5,  3.,  4.5]
    '''
    np.random.seed(seed)
    ranks = rankdata(seq, method='ordinal') # method='ordinal'/'average' ?
    
    random_blocks = np.cumsum(range_ * (2 * np.random.random(divisions + 1) - 1) + np.ones(divisions + 1))
    tresholds = random_blocks[:-1] / random_blocks[-1] * len(seq)
    
    discrete_seq = np.zeros(len(seq), dtype='float64')
    for treshold in tresholds:
        discrete_seq[ranks > treshold] += 1
    return discrete_seq

discretize_vec = np.vectorize(discretize, signature='(n)->(n)', excluded=['divisions', 'range_', 'seed'])

# 2. Read the data (in each rank)

file = "madelon_tiny.csv"
input_ = []
with open(file) as csvfile:
    reader = csv.reader(csvfile, delimiter=',',
                        quoting=csv.QUOTE_NONNUMERIC)
    for row in reader:
        input_.append(row)
        
input_ = np.array(input_, dtype='float64').T[:-1]
data = np.empty(input_.shape, dtype='int32')
data[:-1] = discretize_vec(input_[:-1])
data[-1] = input_[-1:].astype('int32')

labels, counts = np.unique(data[-1], return_counts=True)
n_classes = len(labels)

xi = 0.25
pseudo_counts = xi * counts / np.min(counts)

dim0, dim1 = data[:-1].shape
M = (dim0 - 1) // window + 1
border_cols = range( (M-1) * window, dim0)

if rank == 0:
    print(rank, "dim0 =", dim0, "; dim1 =", dim1)

# 3. More function definitions

def tile_generator(k=k, M=M):
    '''
    Python-generator.
    E.g. output for k=2:
    {0,0}, {0,1}, ..., {0, M-1}, {1,1}, ..., {1,M-1}, ..., {M-1, M-1}
    Go with combinations(range(M), k) to exclude diagonal tuples
    '''        
    return combinations_with_replacement(range(M), k)    

def tuple_generator(tile, window=window, border_cols=border_cols):
    '''
    Map tile into sequence of k-tuples, i.e. fundamental-tiles,
    i.e. elements of the cartesian product of the data-columns
    '''
    index_counts = {index: tile.count(index) for index in tile}
    index_to_cols = lambda index: range(index * window, (index + 1) * window) if index != (M - 1) else border_cols 
    cols_tile = (combinations(index_to_cols(index), count) for (index, count) in index_counts.items())
    return (list(chain.from_iterable(col_indeces)) for col_indeces in product(*cols_tile))

def neg_H(p):
    return p * np.log2(p)

def neg_H_cond(matrix):
    return np.sum(neg_H(matrix)) - np.sum(neg_H(np.sum(matrix, axis=-1)))

def slow_work(tuple_, divisions=divisions, n_classes=n_classes, pseudo_counts=pseudo_counts):
    '''
    tuple_ -> list # dammit...
    Work-function.
    Output: tuple of Information Gains implicitly corresponding to column-indeces in the tuple_
    '''
    # contingency-matrix: begin with pseudo-counts
    contingency_m = np.empty([divisions + 1] * k + [n_classes], dtype='float64')
    for label, pseudo_count in enumerate(pseudo_counts):
        contingency_m[..., label] = pseudo_count
    
    # contingency-matrix: normal counts
    for c_index in data[tuple_ + [-1]].T:
        contingency_m[tuple(c_index)] += 1
    
    IGs = tuple(neg_H_cond(contingency_m) - neg_H_cond(np.sum(contingency_m, axis=i)) for i in range(len(tuple_)))
    return IGs


def record_tuple(tuple_, IGs, records):
    for column, IG in zip(tuple_, IGs):
        if column not in records or IG > records[column][0]:
            records[column] = (IG, tuple_)
            
def record_tile(tile_results, records):
    for column, (IG, tuple_) in tile_results.items():
        if column not in records or IG > records[column][0]:
            records[column] = (IG, tuple_)

# 4 Work loop

if rank == 0:
    final_results = {}
    current_assignements = {rank: (0, None) for rank in range(1,size)}
    
    print(rank, "entering the for loop")
    status = MPI.Status()
    for tile in tile_generator():
        tile_results = comm.recv(status=status)
        record_tile(tile_results, final_results)
        comm.isend(tile, dest=status.source)
        job_count = current_assignements[status.source][0]
        current_assignements[status.source] = (job_count + 1, tile)
        
        #print(rank, "currently:", current_assignements)
    
    print(rank, "Work queue is empty")
    for _ in range(size - 1):
        tile_results = comm.recv(status=status)
        record_tile(tile_results, final_results)
        comm.isend(None, dest=status.source)

    # Save the results to a file
    with open("february0_mpi_results.pkl", "wb") as file:
        pickle.dump(final_results, file)

    print(rank, "says goodbye")
    print(rank, "Elapsed:", MPI.Wtime() - time0, "sec")
    
else:
    comm.send({}, dest=0)
    print(rank, "entering the while loop")
    while True:
        tile = comm.recv(source = 0)
        if tile:
            tile_results = {}
            for tuple_ in tuple_generator(tile):
                
                #IGs = slow_work(tuple_)
                IGs = fast.work_3a(dim1, divisions, data[tuple_[0]], data[tuple_[1]], data[tuple_[2]], n_classes, pseudo_counts, data[-1])
                record_tuple(tuple_, IGs, tile_results)
            comm.isend(tile_results, dest=0)
        else:
            print(rank, "says goodbye")
            break


Overwriting february0_mpi.py


In [56]:
%%bash
mpirun -n 3 python february0_mpi.py

1 entering the while loop
0 dim0 = 30 ; dim1 = 2000
0 entering the for loop
2 entering the while loop
0 Work queue is empty
2 says goodbye
1 says goodbye
0 says goodbye
0 Elapsed: 23.3301741030009 sec


In [57]:
import pandas as pd
import pickle
with open("february0_mpi_results.pkl", "rb") as file:
    final_results = pickle.load(file)

results_mpi_df = pd.DataFrame(final_results).T.rename(columns={0: 'IG_max', 1: 'tuple'})
results_mpi_df[:10]

,IG_max,tuple
0,623.697,"[0, 6, 22]"
1,637.13,"[0, 1, 23]"
2,636.121,"[2, 15, 20]"
3,623.036,"[3, 4, 24]"
4,630.905,"[4, 26, 29]"
5,673.842,"[5, 11, 25]"
6,619.115,"[0, 6, 22]"
7,630.008,"[7, 26, 27]"
8,639.498,"[7, 8, 14]"
9,618.244,"[9, 13, 24]"
